In [121]:
"""
Created on Tuesday 21 Sep 2023
Author: ZAW
"""
##import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

### Read the dataset

In [122]:
df = pd.read_excel('data/list_of_detected_peak_pairs.xlsx',skiprows=2)

In [123]:
df.columns

Index(['No', 'RT (s)', 'Normalized RT (s)', 'm/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity', 'nCharge', 'nTag', 'Unnamed: 9',
       'NovaMT Library No.', 'External Identifier', 'Compound',
       'Mass Error (ppm)', 'RT Error (s)', 'NovaMT Library No..1',
       'External Identifier.1', 'Compound.1', 'Mass Error (ppm).1',
       'RT Error (s).1', 'Zero-reaction', 'One-reaction', 'Two-reaction',
       'Unnamed: 23', 'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25',

### Extract the relevant columns from the dataframe

In [124]:
columns = ['Normalized RT (s)','m/z_light', 'm/z_heavy',
       'Neutral Mass (Da)', 'Intensity','Compound','Compound.1','DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6',
       'DR.7', 'No DR', 'DR.8', 'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12',
       'DR.13', 'DR.14', 'DR.15', 'DR.16', 'No DR.2', 'DR.17', 'DR.18',
       'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21', 'DR.22', 'DR.23',
       'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
       'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15',
       'No DR.16', 'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21',
       'No DR.22', 'No DR.23', 'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27',
       'No DR.28', 'No DR.29']
df = df[columns]

In [125]:
df.drop(df[(df['Compound'].isna() | (df['Compound'] == '')) & (df['Compound.1'].isna() | (df['Compound.1'] == ''))].index, inplace=True)

In [126]:
len(df['Compound.1'])

170

In [128]:
df

,Normalized RT (s),m/z_light,m/z_heavy,Neutral Mass (Da),Intensity,Compound,Compound.1,DR,DR.1,DR.2,...,No DR.20,No DR.21,No DR.22,No DR.23,No DR.24,No DR.25,No DR.26,No DR.27,No DR.28,No DR.29
18,386.3,398.1376,400.1438,150.0524,1449.809490,L-Arabinose,NaN,0.8630,0.7320,0.8620,...,0.803,0.963,1.082,0.830,0.852,0.861,0.702,0.891,1.031,0.800
19,392.5,307.1225,309.1285,59.0373,1152.039064,NaN,Aminoacetaldehyde,1.1760,1.7130,0.8350,...,0.438,0.619,0.669,0.790,0.571,0.729,0.585,0.522,0.664,0.746
41,581.4,394.1433,396.1498,146.0580,1150.063645,NaN,(R)-4-Dehydropantoic acid,0.8670,0.8920,0.8880,...,0.849,0.963,0.942,1.017,0.902,0.917,0.902,0.846,0.999,0.981
46,623.4,394.1433,396.1496,146.0581,1303.804362,NaN,3-Hydroxy-5-oxohexanoic acid,0.9450,0.9830,0.9370,...,0.885,0.961,0.961,1.044,0.876,0.909,0.995,0.848,0.992,1.006
49,683.3,292.1114,294.1179,44.0262,1113.644708,Acetaldehyde,NaN,0.9300,0.9280,0.9610,...,0.821,0.944,0.949,1.030,0.861,0.882,0.889,0.764,0.944,0.945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,1034.5,402.1010,404.1073,168.0426,1044.374071,Vanillic acid,NaN,0.9866,0.2761,0.8764,...,1.133,1.973,1.061,0.602,1.243,0.754,1.723,1.017,1.500,0.780
804,1048.1,372.0905,374.0971,138.0322,1314.406927,4-Hydroxybenzoic acid,NaN,0.5708,1.9410,1.0534,...,1.364,2.177,1.093,0.911,1.785,0.689,1.439,1.161,0.876,0.750
805,1048.3,307.1129,309.1196,146.1091,5248.849898,Lysine (2 tags 2 charges),NaN,0.6710,0.7590,0.4060,...,1.779,0.860,1.121,0.761,1.678,0.581,0.882,1.179,0.644,0.949
809,1080.0,307.1479,309.1547,73.0896,1559.591727,NaN,Diethylamine,0.7140,1.0360,1.0080,...,0.587,0.824,0.964,0.835,0.562,0.651,0.774,0.877,0.985,0.804


In [127]:
len(df)

170

In [114]:
df

,No,RT (s),Normalized RT (s),m/z_light,m/z_heavy,Neutral Mass (Da),Intensity,nCharge,nTag,Unnamed: 9,...,No DR.27,No DR.28,No DR.29,QC,QC.1,QC.2,QC.3,QC.4,QC.5,QC.6
18,K-39,119.1,386.3,398.1376,400.1438,150.0524,1449.809490,1,1,Tier 1,...,0.891,1.031,0.800,0.960,1.000,0.951,0.922,0.893,0.906,0.831
19,K-40,123.6,392.5,307.1225,309.1285,59.0373,1152.039064,1,1,Tier 2,...,0.522,0.664,0.746,0.722,0.811,0.907,0.800,0.838,0.937,0.866
41,K-80,245.8,581.4,394.1433,396.1498,146.0580,1150.063645,1,1,Tier 2,...,0.846,0.999,0.981,0.909,0.937,0.967,1.045,0.981,0.955,0.897
46,K-91,272.6,623.4,394.1433,396.1496,146.0581,1303.804362,1,1,Tier 2,...,0.848,0.992,1.006,0.938,1.004,1.023,0.992,0.958,1.020,0.939
49,K-109,310.0,683.3,292.1114,294.1179,44.0262,1113.644708,1,1,Tier 1,...,0.764,0.944,0.945,0.931,0.977,1.033,0.941,0.916,0.934,0.898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,A-324,475.2,1034.5,402.1010,404.1073,168.0426,1044.374071,1,1,Tier 1,...,1.017,1.500,0.780,1.144,1.157,1.127,1.104,1.043,1.060,1.096
804,A-327,479.3,1048.1,372.0905,374.0971,138.0322,1314.406927,1,1,Tier 1,...,1.161,0.876,0.750,1.214,1.179,1.083,1.058,1.049,1.197,1.065
805,A-328,478.8,1048.3,307.1129,309.1196,146.1091,5248.849898,2,2,Tier 1,...,1.179,0.644,0.949,1.094,1.085,1.052,1.045,1.057,1.063,1.084
809,A-337,491.1,1080.0,307.1479,309.1547,73.0896,1559.591727,1,1,Tier 2,...,0.877,0.985,0.804,1.015,1.119,1.066,0.992,1.053,0.947,1.021


### Extract the specified columns and transpose the DataFrame

In [73]:
# Define the columns to extract
columns = ['Normalized RT (s)', 'm/z_light', 'm/z_heavy', 'Neutral Mass (Da)', 'Intensity',
           'DR', 'DR.1', 'DR.2', 'DR.3', 'DR.4', 'DR.5', 'DR.6', 'DR.7', 'No DR', 'DR.8',
           'DR.9', 'No DR.1', 'DR.10', 'DR.11', 'DR.12', 'DR.13', 'DR.14', 'DR.15', 'DR.16',
           'No DR.2', 'DR.17', 'DR.18', 'DR.19', 'DR.20', 'No DR.3', 'No DR.4', 'DR.21',
           'DR.22', 'DR.23', 'DR.24', 'No DR.5', 'No DR.6', 'No DR.7', 'No DR.8', 'No DR.9',
           'No DR.10', 'No DR.11', 'No DR.12', 'No DR.13', 'No DR.14', 'No DR.15', 'No DR.16',
           'No DR.17', 'No DR.18', 'No DR.19', 'No DR.20', 'No DR.21', 'No DR.22', 'No DR.23',
           'No DR.24', 'No DR.25', 'No DR.26', 'No DR.27', 'No DR.28', 'No DR.29']

In [74]:
# Extract the specified columns
df_selected = df[columns]

In [75]:
# Iterate over each compound name
for compound in df_selected.columns:
    # Extract the data for the compound
    compound_data = df_selected[compound]
    # Create a new row with the compound name as the 'Sample ID' and assign the data to the corresponding columns
    new_row = pd.DataFrame([[compound] + list(compound_data)], columns=['Sample ID'] + list(df['Compound.1']))
    # Append the new row to the transposed DataFrame
    df_transposed = df_transposed.append(new_row, ignore_index=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [48]:
# Print the resulting DataFrame
df_transposed.columns

Index(['Sample ID',         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       ...
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan],
      dtype='object', length=823)

In [44]:
# Create a new DataFrame for transposed data
df_transposed = pd.DataFrame(columns=['Sample ID'] + list(df['Compound.1']))

### Rename the columns to match the desired format

In [13]:
column_names = 